<a id="top"></a>

# Downloading Exoplanet Data

***

# Learning Goals

By the end of this tutorial, you will:

- Understand how MAST makes its exoplanet time-series data accessible.
- Be able to download MAST-hosted data for specific exoplanets.
- Become familiar with the MAST API and two of its services: exo.MAST and astroquery.mast
- Sort MAST data product metadata by attributes (e.g., year, PI).

# Introduction

A number of space-based missions — e.g., the Hubble Space Telescope ([HST](https://archive.stsci.edu/missions-and-data/hst)), the Transiting Exoplanet Survey Satellite ([TESS](https://archive.stsci.edu/missions-and-data/tess)), the Spitzer Space Telescope — have observed exoplanets and their host stars. MAST hosts the data products from these disparate sources, making it possible to aggregate heterogeneous data on a single target within a single interface. 


In this tutorial, we will cover how to explore, sort, and download MAST-hosted exoplanet data using a variety of methods. There are a number of different ways to access MAST's exoplanet data. The first is [exo.MAST](https://exo.mast.stsci.edu/), a "one-stop-shop" for MAST's exoplanet data. A second is [astroquery.MAST](https://astroquery.readthedocs.io/en/latest/mast/mast.html), a module within the [astroquery](https://astroquery.readthedocs.io/en/latest/) framework that can query all MAST data. A third is the [MAST API](https://mast.stsci.edu/api/v0/), which can be used to directly interface with MAST databases. We will walk through all of these methods in turn.


The workflow for this notebook consists of:

- [Querying exoplanet data by name with exo.MAST](#Querying-exoplanet-data-by-name-with-exo.MAST)
    - [Searching the `identifiers` table of exo.MAST to find the canonical name](#Searching-the-`identifiers`-table-of-exo.MAST-to-find-the-canonical-name)
    - [Searching exo.MAST for exoplanet data by name](#Searching-exo.MAST-for-exoplanet-data-by-name)
- [Using astroquery.mast](#Using-astroquery.mast)
- [Directly using the MAST API](#Directly-using-the-MAST-API)
- [Acquire RA and Dec of target](#Acquire-RA-and-Dec-of-target)
- [Perform data search around target RA and Dec](Perform-data-search-around-target-RA-and-Dec)

# Imports 
- ast (Python builtin) to safely evaluate strings.
- json (Python builtin) to load results from HTTP requests.
- numpy to help aggregate our metadata.
- os (Python builtin) to create directories for downloaded data and to manage paths.
- pprint (Python builtin) to neatly print json outputs.
- requests to make HTTPS requests to MAST servers.
- sys (Python builtin) to read our machine's Python version.
- urllib (Python builtin) to submit HTTP requests and interact with the exo.MAST API.
- astroquery.mast to interact with MAST data products.
- astropy to provide data structures that organize our metadata.

In [1]:
import ast
import json
import numpy as np
import os
import pprint
import requests
import sys
import urllib.request
from astroquery.mast import Observations

from astropy.table import Table

from urllib.parse import quote as urlencode

# Querying exoplanet data by name with exo.MAST

[exo.MAST](https://exo.mast.stsci.edu/docs/) is a web service API that is optimized for querying exoplanet data. Our first approach to downloading MAST exoplanet data will use this API.

This portion of the tutorial is inspired by the [exo.MAST documentation](https://exo.mast.stsci.edu/docs/getting_started.html).

## Searching the `identifiers` table of exo.MAST to find the canonical name
Data requests to exo.MAST require the "canonical name" of a planet. If we don't already know the canonical name of a planet, we can determine it using the `identifiers` table. This table is a flexible tool that can be used to cross-reference different names of exoplanets—a helpful feature, given that exoplanets often have multiple aliases across multiple caatlogs. The table is generally able to handle slight permutations to names, such as omitting spaces.

We can query exo.MAST using a planet's name and the exoplanets/identifiers table.

In [2]:
planet_name = 'TIC 86396382 b' # Hebb et al. 2009

We need to safely encode the spaces in this URL for our exo.MAST request. This step is done by replacing all spaces in the name with the "%20" string.

In [3]:
planet_name_formatted = planet_name.replace(' ', '%20')

request_name_string = f'exoplanets/identifiers/?name={planet_name_formatted}'

Now, we append our ``request_name_string`` to the URL that points to the exo.MAST API, forming a complete request URL.

In [4]:
request_name_url = "https://exo.mast.stsci.edu/api/v0.1/" + request_name_string
print(request_name_url)

https://exo.mast.stsci.edu/api/v0.1/exoplanets/identifiers/?name=TIC%2086396382%20b


With our URL assembled, we can now make the HTTPS request using the ``urllib`` package.

In [39]:
names = urllib.request.urlopen(request_name_url).read()
names

b'{"canonicalName":"WASP-12 b","starName":"WASP-12","ra":97.6366416613261,"dec":29.672277777989713,"planetNames":["WASP-12 b","TYC 1891-1178-1 b","Gaia DR1 3435282862461427072 b","2MASS J06303279+2940202 b","2MASS J063032.79+294020.4 b","UCAC2 42216354 b","Bergfors-6 b"],"keplerID":null,"keplerTCE":null,"tessID":86396382,"tessTCE":"TCE_1"}'

The ``b`` before the printed dictionary indicates that these data are currently represented as bytes. To represent the requested data as a string, we need to decode the bytes with the UTF-8 encoding.

In [6]:
dict_str = names.decode("UTF-8")

Now that we've converted the returned bytes into a string, we'd like to evaluate them to case them into a more useful data type: a ``dict``.

However, we need to ensure that the evaluated string is Pythonic to evaluate it. Note that some keys — e.g., ``"keplerID"`` — have values of "null." As a next step, we therefore replace all instances of "null" with "None", which will evaluate to a Python ``None``.

In [7]:
dict_str = dict_str.replace('null', 'None')
dict_str

'{"canonicalName":"WASP-12 b","starName":"WASP-12","ra":97.6366416613261,"dec":29.672277777989713,"planetNames":["WASP-12 b","TYC 1891-1178-1 b","Gaia DR1 3435282862461427072 b","2MASS J06303279+2940202 b","2MASS J063032.79+294020.4 b","UCAC2 42216354 b","Bergfors-6 b"],"keplerID":None,"keplerTCE":None,"tessID":86396382,"tessTCE":"TCE_1"}'

At this point, we can evaluate the string to return a ``dict``. We will use the ``ast.literal_eval`` function to do so. Using ``ast.literal_eval`` instead of ``eval`` is in general better practice, as ``eval`` can accidentally be used to execute malicious code (e.g., see the blog post in the Resources section of this tutorial).

In [8]:
name_matches = ast.literal_eval(dict_str)

In [9]:
name_matches

{'canonicalName': 'WASP-12 b',
 'starName': 'WASP-12',
 'ra': 97.6366416613261,
 'dec': 29.672277777989713,
 'planetNames': ['WASP-12 b',
  'TYC 1891-1178-1 b',
  'Gaia DR1 3435282862461427072 b',
  '2MASS J06303279+2940202 b',
  '2MASS J063032.79+294020.4 b',
  'UCAC2 42216354 b',
  'Bergfors-6 b'],
 'keplerID': None,
 'keplerTCE': None,
 'tessID': 86396382,
 'tessTCE': 'TCE_1'}

``name_matches`` is now a ``dict`` that contains our results. A few important fields:

- ``ra`` and ``dec``: Right ascension and declination of the planet's host star.
- `planetNames`: A ``list`` of names that are used for this planet in different catalogs.

## Searching exo.MAST for exoplanet data by name
With the names for this exoplanet known, we can query exo.MAST for a list of files that are available for it with the following steps:

1. Format a request string
2. Submit the HTTPS request
3. Decode the request's result
4. Replace any "null"s with "None"s
5. Evaluate the string

In [10]:
# use the canonical name
planet_name = name_matches['canonicalName']

# format the canonical name
planet_name_formatted = planet_name.replace(' ', '%20')

# construct request string
request_spectra_string = f'spectra/{planet_name_formatted}/filelist/'
request_spectra_url = "https://exo.mast.stsci.edu/api/v0.1/" + request_spectra_string

# send the request
spectra_result = urllib.request.urlopen(request_spectra_url).read()

# decode the result and make it Pythonic
dict_str = spectra_result.decode("UTF-8")
dict_str = dict_str.replace('null', 'None')

# evaluate the result
spectra = ast.literal_eval(dict_str)
spectra

{'filenames': ['WASP-12b_transmission_Kreidberg2015.txt',
  'WASP-12b_transmission_Sing2016.txt']}

``spectra`` is now a ``dict``. Its key ``filenames`` has a value that is a ``list`` of spectra associated with the submitted planet.

We now know the filenames corresponding to spectra for this planet. Next, we can download one of them, once again following the above steps. We'll hardcode the actual filename here so that the result of this notebook will stay the same if new data are acquired for this target in the future.

For this request, note that the API requires that we query `spectra/{planet_name_formatted}/file/{filename}`. That is, we cannot just query the planet name itself.

In [11]:
# construct request string
filename = spectra['filenames']['WASP-12b_transmission_Kreidberg2015.txt']
request_file_string = f'spectra/{planet_name_formatted}/file/{filename}'
request_file_url = "https://exo.mast.stsci.edu/api/v0.1/" + request_file_string

# send the request
spectra_result = urllib.request.urlopen(request_file_url).read()

# decode the result and make it Pythonic
downloaded_file = spectra_result.decode("UTF-8")
downloaded_file

'# WASP-12b transmission spectrum \n# This file contains:\n# WFC3 G102 - GO 13467 - Kreidberg, et al. 2015, ApJ\n# WFC3 G141 - GO 13467 - Kreidberg, et al. 2015, ApJ\n# Data history:\n# Converted from: - Kreidberg, et al. 2015, ApJ\n# ----------------------------------------------------------------------------------------\n# Wavelength (microns)  Delta Wavelength (microns)  (Rp/Rs)^2   (Rp/Rs)^2 +/-uncertainty\n# ----------------------------------------------------------------------------------------\n         0.8670000    0.02900001   0.01444100   0.00006900  \n         0.9250000    0.02899998   0.01442200   0.00005500  \n         0.9830000    0.02899998   0.01440200   0.00005200  \n         1.0410000    0.02900004   0.01442800   0.00005100  \n         1.0995000    0.02949995   0.01439100   0.00005300  \n         1.1470000    0.03500003   0.01438600   0.00004700  \n         1.2165000    0.03450000   0.01436500   0.00004500  \n         1.2855000    0.03450000   0.01432700   0.00004100 

This time, however, the decoded data cannot be evaluated as a ``dict``. Rather, it is a string of lines that are separated by the new line character ``\n``. We can directly write this string to a file.

In [12]:
with open(filename, 'w') as f:
    f.write(downloaded_file)

Note, however, that not all MAST data products are accessible by this method. (why is this, exactly?)

To download other data products, we turn to other methods.

# Using astroquery.mast

For more MAST data products, we can make use of the ``astroquery.mast`` functionality. This approach requires an additional dependency (the [Astroquery](https://astroquery.readthedocs.io/en/latest/) package).

This portion of the tutorial is inspired by the [astroquery.mast tutorial](https://astroquery.readthedocs.io/en/latest/mast/mast.html).

First, let's search for all MAST data products for a single exoplanet.

In [13]:
search_radius = ".02 deg"

planet_name = 'WASP-12 b'
obs_table = Observations.query_object(planet_name,radius=search_radius)

# only print the first 10 results for clarity
print(obs_table[:10])  

intentType obs_collection provenance_name instrument_name project filters ... dataRights mtFlag srcDen  obsid   distance
---------- -------------- --------------- --------------- ------- ------- ... ---------- ------ ------ -------- --------
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 27266912      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 65432622      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 68942182      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 71308547      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 71741843      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 27242856      0.0
   science           TESS       

In [14]:
obs_table.columns

<TableColumns names=('intentType','obs_collection','provenance_name','instrument_name','project','filters','wavelength_region','target_name','target_classification','obs_id','s_ra','s_dec','dataproduct_type','proposal_pi','calib_level','t_min','t_max','t_exptime','em_min','em_max','obs_title','t_obs_release','proposal_id','proposal_type','sequence_number','s_region','jpegURL','dataURL','dataRights','mtFlag','srcDen','obsid','distance')>

Let's sort these observations by the proposing PI (principal investigator) and filter out extraneous columns.

In [15]:
obs_table.sort('proposal_pi')

In [16]:
print(obs_table[['proposal_pi', 'provenance_name', 'dataproduct_type']])

  proposal_pi   provenance_name dataproduct_type
--------------- --------------- ----------------
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean             HLA            image
           Bean     

If we're interested in a specific instrument, we can next see which instruments were used to observe this target.

In [17]:
print(np.unique(obs_table['provenance_name']))

provenance_name
---------------
            3PI
         CALCOS
        CALSTIS
         CALWF3
          CDIPS
            HAP
        HAP-MVM
            HLA
             --
            QLP
           SPOC
   SSC Pipeline
      TESS-SPOC


Great! In just a few lines, we've collected the metadata for many observations of this target into an Astropy ``Table``. Next, let's see what data products are available for the most recent Quick Look Pipeline ([QLP](https://tess.mit.edu/qlp/)) data. The QLP is a pipeline from the TESS Science Office that processes TESS full-frame images to search for transiting exoplanets. Because we'll sort these data by time, the most recent data is the last entry, which we can access with the ``-1`` index.

In [18]:
obs_table_qlp = obs_table[obs_table['provenance_name']=='QLP']
obs_table_qlp.sort('t_min')

recent_index = -1
data_products_by_obs = Observations.get_product_list(obs_table_qlp[recent_index])
print(data_products_by_obs) 

 obsID   obs_collection dataproduct_type                         obs_id                        description ...  size parent_obsid dataRights calib_level
-------- -------------- ---------------- ----------------------------------------------------- ----------- ... ----- ------------ ---------- -----------
36811427           HLSP       timeseries hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc        FITS ... 77760     36811427     PUBLIC           4
36811427           HLSP       timeseries hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc        Text ... 48016     36811427     PUBLIC           4


There are two timeseries data products. Let's download the first one. 

In [19]:
data_products_by_obs.columns

<TableColumns names=('obsID','obs_collection','dataproduct_type','obs_id','description','type','dataURI','productType','productGroupDescription','productSubGroupDescription','productDocumentationURL','project','prvversion','proposal_id','productFilename','size','parent_obsid','dataRights','calib_level')>

In [20]:
# select first observation collection
obs_collection = data_products_by_obs['obs_collection'][0]
obs_id = data_products_by_obs['obs_id'][0]

# query and get product list for this collection
single_obs = Observations.query_criteria(obs_collection=obs_collection, obs_id=obs_id)
data_products = Observations.get_product_list(single_obs)

# download data products
manifest = Observations.download_products(data_products, productType="SCIENCE")

INFO: Found cached file ./mastDownload/HLSP/hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc/hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc.fits with expected size 77760. [astroquery.query]
INFO: Found cached file ./mastDownload/HLSP/hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc/hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc.txt with expected size 48016. [astroquery.query]


Let's take a closer look at the list of downloaded files, known as the "manifest."

In [21]:
print(manifest)

                                                             Local Path                                                               Status  Message URL 
------------------------------------------------------------------------------------------------------------------------------------ -------- ------- ----
./mastDownload/HLSP/hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc/hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc.fits COMPLETE    None None
 ./mastDownload/HLSP/hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc/hlsp_qlp_tess_ffi_s0020-0000000086396382_tess_v01_llc.txt COMPLETE    None None


We've now successfully downloaded MAST data using the ``astroquery.mast`` API.

# Directly using the MAST API
This final approach is a bit more hands-on and requires more code, but it allows for the most flexibility. Additionally, it provides the most insight into what's going on "under the hood" with MAST data requests. This approach requires the [Astropy](https://www.astropy.org/) and [NumPy](https://numpy.org/) dependencies.

This portion of the tutorial is inspired by the general [MAST API tutorial](https://mast.stsci.edu/api/v0/MastApiTutorial.html).

## Acquire RA and Dec of target
With the MAST API, we'll need the sky coordinates of our target. Our first step is to therefore find the right ascension and declintation of our target.

We'll start off by initializing [``pprint``](https://docs.python.org/3/library/pprint.html). This module prints arbitrary python data in a more readable way. Without it, json data in particular can appear compact and difficult to read.

In [22]:
pp = pprint.PrettyPrinter(indent=4)

Sticking with our previous example, let's look at the planet WASP-12 b. First, we need to format our request to the MAST resolver — the service that matches requests to MAST data.

In [23]:
object_of_interest = 'WASP-12 b'

resolver_request = {'service':'Mast.Name.Lookup',
                     'params':{'input':object_of_interest,
                               'format':'json'},
                     }

# encoding the request as a json string
req_string = json.dumps(resolver_request)
req_string = urlencode(req_string)

Next, we set the header variables needed to make the request.

In [24]:
# retrieve our system's Python version for the request. 
version = ".".join(map(str, sys.version_info[:3]))

# create HTTP Header Variables
headers = {"Content-type": "application/x-www-form-urlencoded",
           "Accept": "text/plain",
           "User-agent":"python-requests/"+version}


Now, we construct our request URL and perform our HTTP request with the ``requests`` package.

In [25]:
# base API url
request_url='https://mast.stsci.edu/api/v0/invoke'    

# perform the HTTP request
resp = requests.post(request_url, data="request="+req_string, headers=headers)

The response must be decoded into a string, as in the previous examples.

In [26]:
resolved_object = resp.json()

pp.pprint(resolved_object)

{   'resolvedCoordinate': [   {   'cached': False,
                                  'canonicalName': 'WASP-12 b',
                                  'decl': 29.672277777989713,
                                  'ra': 97.6366416613261,
                                  'resolver': 'EXO',
                                  'resolverTime': 34,
                                  'searchRadius': 0.000333,
                                  'searchString': 'wasp-12 b'}],
    'status': ''}


Parsing apart some of the output:
- the *cached* field denotes whether this result has already been saved on this device.
- the *canonicalName* field denotes the default name of the planet.
- the *decl* (float) field denotes the declination of the resolved coordinate.
- the *ra* (float) field denotes the right ascention of the resolved coordinate.
- the *searchRadius* field denotes the raidius of the search.

See the [documentation](https://mast.stsci.edu/api/v0/) for further information.

Now that we've resolved our target, let's save its coordinates as (float) variables — we'll need them later on.

In [27]:
obj_ra = resolved_object['resolvedCoordinate'][0]['ra']
obj_dec = resolved_object['resolvedCoordinate'][0]['decl']

obj_ra, obj_dec

(97.6366416613261, 29.672277777989713)

## Perform data search around target RA and Dec

With the coordinates of the object now known, we can run a [Mast.Caom.Cone](https://mast.stsci.edu/api/v0/pyex.html#MastCaomConePy) query to retrieve metadata on all MAST data around this coordinate.

As `params`, we'll pass the object's position (in right ascension and declination) and the desired rearch radius in degrees.

Together, the `page` and `pagesize` parameters control how many results are returned. A user can either set a `pagesize` that is large enough to return all results in a single page, or a user can set a smaller `pagesize` and step through results in successive requests by requesting different `page` values.

In [28]:
mast_request = {'service':'Mast.Caom.Cone',
                'params':{'ra':obj_ra,
                          'dec':obj_dec,
                          'radius':0.2},
                'format':'json',
                'pagesize':2000,
                'page':1,
                'removenullcolumns':True,
                'removecache':True}


# encoding the request as a json string
req_string = json.dumps(mast_request)
req_string = urlencode(req_string)

# perform the HTTP request
resp = requests.post(request_url, data="request="+req_string, headers=headers)

# decode the HTTP result
mast_data = resp.json()


print(mast_data.keys())
print("Query status:",mast_data['status'])

dict_keys(['status', 'msg', 'data', 'fields', 'paging'])
Query status: COMPLETE


Let's take a look at the first returned data entry.

In [29]:
pp.pprint(mast_data['data'][0])

{   '_selected_': None,
    'calib_level': 3,
    'dataRights': 'PUBLIC',
    'dataURL': None,
    'dataproduct_type': 'image',
    'distance': 0,
    'em_max': 1000,
    'em_min': 600,
    'filters': 'TESS',
    'instrument_name': 'Photometer',
    'intentType': 'science',
    'jpegURL': None,
    'mtFlag': False,
    'obs_collection': 'TESS',
    'obs_id': 'tess-s0020-1-3',
    'obs_title': None,
    'obsid': 27266912,
    'project': 'TESS',
    'proposal_id': 'N/A',
    'proposal_pi': 'Ricker, George',
    'proposal_type': None,
    'provenance_name': 'SPOC',
    's_dec': 34.91458576307677,
    's_ra': 101.64458792443914,
    's_region': 'POLYGON 94.61647600 29.34512900 93.87837900 40.81682500 '
                '109.96143000 40.20664100 108.07068800 28.39783100 94.61647600 '
                '29.34512900 ',
    'sequence_number': 20,
    'srcDen': None,
    't_exptime': 1425.599406,
    't_max': 58868.3173308,
    't_min': 58842.00512903,
    't_obs_release': 58893,
    'target_class

There's a lot of metadata here, and it's a bit hard to understand all at once. To make things a bit more digestible, we can create an Astropy ``Table``.

To do so, we need to loop through each ``field`` in ``mast_data``. These ``field``s will become the columns of our ``Table``. The ``field``s also have associated data types that are contained in ``mast_data``; however, we'll need to convert the names of these data types into Pythonic data types. For example, the data type for the field ``indentType`` is ``'string'``, but the corresponding Python data type is ``str``, so we'll need to take this into account when creating the ``Table``.

In [30]:
mast_data_table = Table()

col_type_list = [(field['name'], field['type']) for field in mast_data['fields']]

for col, atype in col_type_list:
    
    # recast data types
    if atype=="string":
        atype="str"
    if atype=="boolean":
        atype="bool"
    
    # create column in Table corresponding to the mast_data field
    mast_data_table[col] = np.array([x.get(col, None) for x in mast_data['data']],dtype=atype)
    
print(mast_data_table)

intentType obs_collection provenance_name instrument_name project filters wavelength_region ... dataRights mtFlag srcDen  obsid       distance     _selected_
---------- -------------- --------------- --------------- ------- ------- ----------------- ... ---------- ------ ------ -------- ---------------- ----------
   science           TESS            SPOC      Photometer    TESS    TESS           Optical ...     PUBLIC  False    nan 27266912              0.0      False
   science           TESS            SPOC      Photometer    TESS    TESS           Optical ...     PUBLIC  False    nan 65432622              0.0      False
   science           TESS            SPOC      Photometer    TESS    TESS           Optical ...     PUBLIC  False    nan 68942182              0.0      False
   science           TESS            SPOC      Photometer    TESS    TESS           Optical ...     PUBLIC  False    nan 71308547              0.0      False
   science           TESS            SPOC      Photo

Note that, because we set our ``pagesize`` to 2000, we will only see the first 2000 rows returned here. Other MAST data exists that meets our criteria, but we would need to alter the ``pagesize`` value to access them.

With our metadata all acquired, we can now sort it based on, e.g., start date.

In [31]:
mast_data_table.sort('t_min')

In [32]:
print(mast_data_table)

intentType obs_collection provenance_name instrument_name project filters wavelength_region ... dataRights mtFlag srcDen  obsid       distance     _selected_
---------- -------------- --------------- --------------- ------- ------- ----------------- ... ---------- ------ ------ -------- ---------------- ----------
   science    SPITZER_SHA    SSC Pipeline            IRAC    None   IRAC2          Infrared ...     PUBLIC  False    nan  1692417              0.0      False
   science    SPITZER_SHA    SSC Pipeline            IRAC    None   IRAC4          Infrared ...     PUBLIC  False    nan  1692417              0.0      False
   science    SPITZER_SHA    SSC Pipeline            IRAC    None   IRAC2          Infrared ...     PUBLIC  False    nan  1692417              0.0      False
   science    SPITZER_SHA    SSC Pipeline            IRAC    None   IRAC4          Infrared ...     PUBLIC  False    nan  1692417              0.0      False
   science    SPITZER_SHA    SSC Pipeline           

Let's get the most recent Spitzer data product. Because these data are sorted by time, the most recent data is the last entry, which we can access with the ``-1`` index.

In [33]:
recent_index = -1
interesting_observation = mast_data_table[mast_data_table["obs_collection"] == "SPITZER_SHA"][recent_index]
print("Observation:",
      [interesting_observation[x] for x in ['dataproduct_type', 'obs_collection', 'instrument_name']])

Observation: ['image', 'SPITZER_SHA', 'IRAC']


It appears that the latest Spitzer data for this target was taken with IRAC (Infrared Array Camera).

We can search MAST using the ``obsid`` of this observation to determine how many data products are associated with this observation. This is similar to our previous MAST API request, except now we query by a specific observation ID instead of a position (RA and Dec).

In [34]:
obsid = interesting_observation['obsid']

product_request = {'service':'Mast.Caom.Products',
                  'params':{'obsid':obsid},
                  'format':'json',
                  'pagesize':100,
                  'page':1}   

# encoding the request as a json string
req_string = json.dumps(product_request)
req_string = urlencode(req_string)

# perform the HTTP request
resp = requests.post(request_url, data="request="+req_string, headers=headers)

# dfrecode the HTTP result
obs_products = resp.json()

print("Number of data products:", len(obs_products["data"]))
print("Product information column names:")
pp.pprint(obs_products['fields'])

Number of data products: 100
Product information column names:
[   {'name': 'obsID', 'type': 'string'},
    {'name': 'obs_collection', 'type': 'string'},
    {'name': 'dataproduct_type', 'type': 'string'},
    {'name': 'obs_id', 'type': 'string'},
    {'name': 'description', 'type': 'string'},
    {'name': 'type', 'type': 'string'},
    {'name': 'dataURI', 'type': 'string'},
    {'name': 'productType', 'type': 'string'},
    {'name': 'productGroupDescription', 'type': 'string'},
    {'name': 'productSubGroupDescription', 'type': 'string'},
    {'name': 'productDocumentationURL', 'type': 'string'},
    {'name': 'project', 'type': 'string'},
    {'name': 'prvversion', 'type': 'string'},
    {'name': 'proposal_id', 'type': 'string'},
    {'name': 'productFilename', 'type': 'string'},
    {'name': 'size', 'type': 'int'},
    {'name': 'parent_obsid', 'type': 'string'},
    {'name': 'dataRights', 'type': 'string'},
    {'name': 'calib_level', 'type': 'int'},
    {'name': '_selected_', 'type'

We can also see what *types* these data products are. 'SCIENCE' products are meant to be used for analysis, whereas 'AUXILIARY' and 'NOISE' products are supplemental (e.g., used for calibration).

In [35]:
file_types = [x.get('productType',"") for x in obs_products["data"]]

print(np.unique(file_types))

['AUXILIARY' 'NOISE' 'SCIENCE']


We can place these results in a table as well, restricting ourselves to the science products. As before, we need to be careful about recasting the listed data types to Pythonic types.

In [36]:
sci_prod_arr = [x for x in obs_products['data'] if x.get("productType", None) == 'SCIENCE']
science_products = Table()

col_type_list = [(field['name'], field['type']) for field in obs_products['fields']]

for col, atype in col_type_list:
    
    # recast data types
    if atype=="string":
        atype="str"
    if atype=="boolean":
        atype="bool"
    if atype == "int":
        atype = "float" # array may contain nan values, and they do not exist in numpy integer arrays
        
    # Create column in Table corresponding to the mast_data field
    science_products[col] = np.array([x.get(col,None) for x in sci_prod_arr],dtype=atype)

print("Number of science products:",len(science_products))
print(science_products)

Number of science products: 8
 obsID  obs_collection dataproduct_type    obs_id    description  type ...   size   parent_obsid dataRights calib_level _selected_
------- -------------- ---------------- ------------ ------------ ---- ... -------- ------------ ---------- ----------- ----------
1696175    SPITZER_SHA            image 000001AF1000 BCD (MJy/sr)    S ... 290880.0      1696175     PUBLIC         2.0      False
1696175    SPITZER_SHA            image 000001AF1000 BCD (MJy/sr)    S ... 290880.0      1696175     PUBLIC         2.0      False
1696175    SPITZER_SHA            image 000001AF1000 BCD (MJy/sr)    S ... 290880.0      1696175     PUBLIC         2.0      False
1696175    SPITZER_SHA            image 000001AF1000 BCD (MJy/sr)    S ... 290880.0      1696175     PUBLIC         2.0      False
1696175    SPITZER_SHA            image 000001AF1000 BCD (MJy/sr)    S ... 290880.0      1696175     PUBLIC         2.0      False
1696175    SPITZER_SHA            image 000001AF1000 

Next, let's download these data products using the [requests](https://requests.readthedocs.io/en/latest/) package. 

Each row in the `science_products` object refers to a different data product, so we'll iterate through the object to download each product. For each data product, we'll need to send a specific request that includes that data product's uniform resource identifier (URI). We'll then save the response to that request to a file and check that it has been saved correctly.

In [37]:
download_url = 'https://mast.stsci.edu/api/v0.1/Download/file?'

for row in science_products:     

    # Make file path
    out_path = os.path.join("mastFiles", row['obs_collection'], row['obs_id'])
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    out_path = os.path.join(out_path, os.path.basename(row['productFilename']))
        
    # Download the data
    payload = {"uri":row['dataURI']}
    resp = requests.get(download_url, params=payload)
    
    # Save to file
    with open(out_path,'wb') as f:
        f.write(resp.content)
        
    # Check for file 
    if not os.path.isfile(out_path):
        print("ERROR: " + out_path + " failed to download.")
    else:
        print("COMPLETE: ", out_path)

COMPLETE:  mastFiles/SPITZER_SHA/000001AF1000/SPITZER_I1_28250112_0000_0000_6_bcd.fits
COMPLETE:  mastFiles/SPITZER_SHA/000001AF1000/SPITZER_I1_28250112_0001_0000_6_bcd.fits
COMPLETE:  mastFiles/SPITZER_SHA/000001AF1000/SPITZER_I1_28250112_0002_0000_6_bcd.fits
COMPLETE:  mastFiles/SPITZER_SHA/000001AF1000/SPITZER_I1_28250112_0003_0000_6_bcd.fits
COMPLETE:  mastFiles/SPITZER_SHA/000001AF1000/SPITZER_I1_28250112_0004_0000_6_bcd.fits
COMPLETE:  mastFiles/SPITZER_SHA/000001AF1000/SPITZER_I1_28250112_0005_0000_6_bcd.fits
COMPLETE:  mastFiles/SPITZER_SHA/000001AF1000/SPITZER_I1_28250112_0006_0000_6_bcd.fits
COMPLETE:  mastFiles/SPITZER_SHA/000001AF1000/SPITZER_I1_28250112_0007_0000_6_bcd.fits


We can check that these data files have been downloaded correctly by checking the directory that the `out_path`s are downloaded to.

In [38]:
ls mastFiles/SPITZER_SHA/000001AF1000/

SPITZER_I1_28250112_0000_0000_6_bcd.fits
SPITZER_I1_28250112_0001_0000_6_bcd.fits
SPITZER_I1_28250112_0002_0000_6_bcd.fits
SPITZER_I1_28250112_0003_0000_6_bcd.fits
SPITZER_I1_28250112_0004_0000_6_bcd.fits
SPITZER_I1_28250112_0005_0000_6_bcd.fits
SPITZER_I1_28250112_0006_0000_6_bcd.fits
SPITZER_I1_28250112_0007_0000_6_bcd.fits


Great! We've successfully downloaded the data for this planet via MAST.

# Exercises
1. Use the MAST API to sort Swift data on WASP-12 b by PI and download it.
2. Query the latest light curve of WASP-12 b with `astroquery.mast`. 
3. Query data for the planet BD+47 2936 b using the `exo.MAST` API.

# Additional Resources
- An [introduction to HTTP GET requests](https://www.ibm.com/docs/en/cics-ts/5.3?topic=protocol-http-requests).
- Primers on exoplanet spectral data types ([Deming, Louie, and Sheets 2018](https://iopscience.iop.org/article/10.1088/1538-3873/aae5c5/meta?casa_token=253HfRr4kyYAAAAA:C0CtfuH4Um2l4Kul5O3tajY2TolSVuXi8fGj48bzSlmJIuvPmeYkb1yXtd10MOjwPqJokDpNvv4)) and on lightcurves ([Winn 2010](https://books.google.com/books?hl=en&lr=&id=VlSVmxgPgGYC&oi=fnd&pg=PA55&dq=exoplanet+transits&ots=-sl8U--Mws&sig=lvBd93ioa2YDxehtpxSx6nL82UA#v=onepage&q=exoplanet%20transits&f=false))
- Information on the [TESS Mission](https://tess.mit.edu/).
- A [neat blog post](https://nedbatchelder.com/blog/201206/eval_really_is_dangerous.html) on why `literal_eval` is preferred to `eval`.
- [Prettyprint documentation](https://docs.python.org/3/library/pprint.html)

- [Gaudi, B. Scott, et al. "A giant planet undergoing extreme-ultraviolet irradiation by its hot massive-star host." Nature 546.7659 (2017): 514-518.](https://www.nature.com/articles/nature22392)
- [Bakos, G. Á., et al. "HAT-P-11b: A super-Neptune planet transiting a bright K star in the Kepler field." The Astrophysical Journal 710.2 (2010): 1724.](https://iopscience.iop.org/article/10.1088/0004-637X/710/2/1724/meta)
- [exo.MAST tutorial](https://exo.mast.stsci.edu/docs/getting_started.html#resolving-exoplanets)
- [astroquery.MAST documentation](https://astroquery.readthedocs.io/en/latest/mast/mast.html)
- [MAST API documentation](https://mast.stsci.edu/api/v0/MastApiTutorial.html)

# About this Notebook

**Author**: Arjun B. Savel (asavel@umd.edu).

**Last updated**: 2022-09-02

# Citations
If you use `astropy`, `astroquery`, or `numpy` for published research, please cite the
authors. Follow these links for more information about citing `astropy`,
`astroquery`, and `numpy`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `astroquery`](https://astroquery.readthedocs.io/en/latest/#astroquery)
* [Citing `numpy`](https://numpy.org/citing-numpy/)


***

[Top of Page](#top)<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>
